In [5]:
import pandas as pd
import altair as alt

# Transforming Data

The bread and butter of a data workflow is cleaning and preparation, taking raw datasets and transforming them into a useful form.

Today, we'll use some prescription cost data from the NHS to make a chart of the top 50 most used Cardiovascular substances.

</br></br></br></br>


## Introducing Tools: Pandas

The first tool we'll use today is `Pandas`, a Python library used to work with datasets. It provides access to `DataFrames` - tables we analyse with code.

Python already has a few built in data structures, for example lists and dictionaries:

</br></br></br>

In [48]:
london = {
    "name": "London",
    "population": 8308369,
    "area": 1572
} # This is an example of a dictionary

locations = [
    {
        "name": "London",
        "population": 8_982_000,
        "area": 606
    },
    {
        "name": "Newport",
        "population": 128_060,
        "area": 32.52
    },
    {
        "name": "Darlington",
        "population": 93_015,
        "area": 7.62
    },

]


<br>
<br>
<br>
<br>
Which we can turn into Pandas `DataFrames`

In [49]:
df = pd.DataFrame(locations)
df

,name,population,area
0,London,8982000,606.00
1,Newport,128060,32.52
2,Darlington,93015,7.62


<br>
<br>
<br></br></br>

<br>

and manipulate in different ways.

For example, we can add a density column:

In [50]:
df['density'] = df['population'] / df['area']
df

,name,population,area,density
0,London,8982000,606.00,14821.782178
1,Newport,128060,32.52,3937.884379
2,Darlington,93015,7.62,12206.692913


In [51]:
sorted_df = df.sort_values(by="density", ascending=False)
sorted_df

,name,population,area,density
0,London,8982000,606.00,14821.782178
2,Darlington,93015,7.62,12206.692913
1,Newport,128060,32.52,3937.884379


# Using Your Own Data: Prescription Cost Analysis

Let's use this dataset of prescription costs to make a scatter chart of the most prescribed substances in the category 'Cardiovascular System'/

We will:

1. Load in the data, from an `.xlsx` document with lots of sheets.
2. Rename and restructure our fields.
3. Filter our data.
4. Export it to a csv for use with Vega-lite.

</br></br></br>


## Loading our Data

I have uploaded the `xlsx` spreedsheet to GitHub but you could easily use a file from your computer.

We want data just from the sheet 'Chemical_Substances'.

In [35]:
df = pd.read_excel("https://github.com/EconomicsObservatory/courses/raw/main/5/sample_data/pca_summary_tables_2023_24_v001.xlsx", sheet_name="Chemical_Substances", skiprows=3)

Let's take a look at the DataFrame.

In [36]:
df.head(5)

,Financial Year,BNF Chemical Substance Code,BNF Chemical Substance Name,BNF Paragraph Code,BNF Paragraph Name,BNF Section Code,BNF Section Name,BNF Chapter Code,BNF Chapter Name,Total Items,Total Cost (£),Cost Per Item (£)
0,2023/2024,0101010C0,Aluminium hydroxide,10101,Antacids and simeticone,101,Dyspepsia and gastro-oesophageal reflux disease,1,Gastro-Intestinal System,1,13.71,13.710000
1,2023/2024,0101010F0,Magnesium carbonate,10101,Antacids and simeticone,101,Dyspepsia and gastro-oesophageal reflux disease,1,Gastro-Intestinal System,398,59544.37,149.608970
2,2023/2024,0101010G0,Co-magaldrox (Magnesium/aluminium hydroxide),10101,Antacids and simeticone,101,Dyspepsia and gastro-oesophageal reflux disease,1,Gastro-Intestinal System,28198,144222.89,5.114650
3,2023/2024,0101010I0,Magnesium oxide,10101,Antacids and simeticone,101,Dyspepsia and gastro-oesophageal reflux disease,1,Gastro-Intestinal System,4439,421785.42,95.018117
4,2023/2024,0101010J0,Magnesium trisilicate,10101,Antacids and simeticone,101,Dyspepsia and gastro-oesophageal reflux disease,1,Gastro-Intestinal System,1068,6383.25,5.976826


</br> </br>

## Renaming and Restructuring

We have lots of columns in our data, not all of which we need. Let's rename the columns and only keep the ones we want.

In [37]:
df = df.rename(columns={ # specifying that we want to rename columns
    "BNF Chemical Substance Name": "substance_name",
    "BNF Paragraph Name": "paragraph_name",
    "BNF Section Name": "section_name",
    "BNF Chapter Name": "chapter_name",
    "Total Items": "total_items",
    "Total Cost (£)": "total_cost",
    "Cost Per Item (£)": "cost_per_item",
})

df = df[['subtance_name', 'paragraph_name', 'section_name', 'chapter_name', 'total_items', 'total_cost', 'cost_per_item']] # specifying just the columns we want to keep

</br></br></br>

## Filtering our Data

We only want data for the 'Cardiovascular System' chapter. Let's filter using `df.query`.

In [39]:
df = df.query("chapter_name == 'Cardiovascular System'")
df

,subtance_name,paragraph_name,section_name,chapter_name,total_items,total_cost,cost_per_item
111,Digoxin,Cardiac glycosides,Positive inotropic drugs,Cardiovascular System,2561506,8415584.68,3.285405
112,Enoximone,Phosphodiesterase Type-3 inhibitors,Positive inotropic drugs,Cardiovascular System,1,15.02,15.020000
113,Bendroflumethiazide,Thiazides and related diuretics,Diuretics,Cardiovascular System,5895469,4962387.04,0.841729
114,Chlorothiazide,Thiazides and related diuretics,Diuretics,Cardiovascular System,2314,106645.49,46.087074
115,Chlortalidone,Thiazides and related diuretics,Diuretics,Cardiovascular System,15546,1154916.51,74.290268
...,...,...,...,...,...,...,...
286,Gemfibrozil,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,6040,198960.84,32.940536
287,Nicotinic acid,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,4,133.32,33.330000
288,Pravastatin sodium,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,2233790,4518365.47,2.022735
289,Simvastatin,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,13666725,14597488.57,1.068104


We have 180 rows. Let's just keep the top 50 with the highest item count, which we can do by sorting and taking the top 50 rows:

In [42]:
df = df.sort_values("total_items", ascending=False)
df

,subtance_name,paragraph_name,section_name,chapter_name,total_items,total_cost,cost_per_item
279,Atorvastatin,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,65475356,1.517661e+08,2.317912
217,Amlodipine,Calcium-channel blockers,"Nitrates, calcium-channel blockers & other ant...",Cardiovascular System,37421399,3.695931e+07,0.987652
191,Ramipril,Renin-angiotensin system drugs,Hypertension and heart failure,Cardiovascular System,33447735,4.859265e+07,1.452794
151,Bisoprolol fumarate,Beta-adrenoceptor blocking drugs,Beta-adrenoceptor blocking drugs,Cardiovascular System,28799455,2.899027e+07,1.006626
259,Aspirin,Antiplatelet drugs,Antiplatelet drugs,Cardiovascular System,20199764,1.858923e+07,0.920270
...,...,...,...,...,...,...,...
173,Indoramin,Alpha-adrenoceptor blocking drugs,Hypertension and heart failure,Cardiovascular System,1,4.017000e+01,40.170000
112,Enoximone,Phosphodiesterase Type-3 inhibitors,Positive inotropic drugs,Cardiovascular System,1,1.502000e+01,15.020000
131,Co-triamterzide(Triamterene/hydrochlorothiazide),Potassium sparing diuretics and compounds,Diuretics,Cardiovascular System,1,9.500000e-01,0.950000
240,Ephedrine hydrochloride,Vasoconstrictor sympathomimetics,Sympathomimetics and other vasoconstrictor drugs,Cardiovascular System,1,1.909000e+01,19.090000


</br> </br>
Our data is sorted - let's just take the top 50 rows.

In [43]:
df = df.head(50)

In [44]:
df

,subtance_name,paragraph_name,section_name,chapter_name,total_items,total_cost,cost_per_item
279,Atorvastatin,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,65475356,1.517661e+08,2.317912
217,Amlodipine,Calcium-channel blockers,"Nitrates, calcium-channel blockers & other ant...",Cardiovascular System,37421399,3.695931e+07,0.987652
191,Ramipril,Renin-angiotensin system drugs,Hypertension and heart failure,Cardiovascular System,33447735,4.859265e+07,1.452794
151,Bisoprolol fumarate,Beta-adrenoceptor blocking drugs,Beta-adrenoceptor blocking drugs,Cardiovascular System,28799455,2.899027e+07,1.006626
259,Aspirin,Antiplatelet drugs,Antiplatelet drugs,Cardiovascular System,20199764,1.858923e+07,0.920270
289,Simvastatin,Lipid-regulating drugs,Lipid-regulating drugs,Cardiovascular System,13666725,1.459749e+07,1.068104
205,Losartan potassium,Renin-angiotensin system drugs,Hypertension and heart failure,Cardiovascular System,11912296,1.589276e+07,1.334147
260,Clopidogrel,Antiplatelet drugs,Antiplatelet drugs,Cardiovascular System,11352719,1.459731e+07,1.285799
121,Furosemide,Loop diuretics,Diuretics,Cardiovascular System,11071618,1.083850e+07,0.978945
258,Apixaban,Oral anticoagulants,Anticoagulants and protamine,Cardiovascular System,8470475,2.388879e+08,28.202419


</br> </br>

# Exporting the Data

Finally let's save the cleaned and filtered data as a `CSV` to use with Vega-lite.

In [47]:
df.to_csv("sample_data/cardiovascular_system_substances.csv", index=False)